## Retriever 담당  
목표  
이전에 임베딩 해두었던 output_chunks_with_embeddings.json 사용하여 4 가지 리트리버 비교하기   

1. Dense(벡터): FAISS + E5 임베딩(이미 계산된 것 사용, 재임베딩 없음)

2. TF-IDF: scikit-learn

3. BM25: rank_bm25

4. Multi-Query Dense: 질의를 여러 개로 확장해서 Dense 검색 (간단 LLM 없이 규칙 기반/옵션으로 T5 사용 가능)

현재 내 담당 : 리트리버 구현 & 결과 출력 (기둥 잡기)

---

## 참고할 내용  
### => 팀원들 공유 내용은 "리트리버 실험 자동화 패키지"라고 판단.  

.env = huggingface api key 등 환경 변수  
rag_with_retriever.pu = langchain + 리트리버 + 생성 모델 (RAG 파이프라인 구현 코드)  
rag_Retriever_experiment,ipynb = 실험 자동화 노트북 (retriever 종류만 바꿔 돌려보는 용도)  
experiment_summary,py = 실험 결과 한 번에 요약  


=> 일단은 직접 리트리버 구현한 코드로 실험 진행 후 csv에 저장하는 형태로 진행하면 내부 로직을 더 이해하기 쉽지 않을까 하는 생각에 해당 방식으로 진행하였습니다.  
(이후 문제가 있다면 이전 공유해주신 rag 체인으로 결과/요약 진행할 것 같습니다.)

---

- 현재 진행   
전처리 -> 청킹 -> 임베딩 -> json 저장 진행  
이후   

### 드라이브 마운트 & 경로

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# 파일경로 *각자 사용할 때 수정하면 될 것 같습니다.
EMB_FILE = "/content/drive/MyDrive/output_chunks_with_embeddings.json"

# 확인
import os
print("exists:", os.path.exists(EMB_FILE), EMB_FILE)

Mounted at /content/drive
exists: True /content/drive/MyDrive/output_chunks_with_embeddings.json


** 필요한 패키지 설치 **

In [4]:
!pip -q install faiss-cpu scikit-learn rank-bm25 sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00


## 데이터 로드 (임베딩 json 파일 ->)

In [5]:
import json, numpy as np

with open(EMB_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [d.get("text","") for d in data if isinstance(d.get("text"), str)]
embs  = np.asarray([d["embedding"] for d in data if isinstance(d.get("embedding"), list)], dtype="float32")
meta  = [{
    "doc_id": d.get("doc_id"),
    "chunk_index": d.get("chunk_index"),
    "filename": d.get("filename"),
    "folder": d.get("folder"),
} for d in data]

print(f"청크 수: {len(texts)} | 임베딩 shape: {embs.shape}")
print("샘플:", texts[0][:150].replace("\n"," "))

청크 수: 1057 | 임베딩 shape: (1057, 1024)
샘플: 양도소득세부과처분취소 [수원지법 2007. 11. 28. 선고 2007구합3771 판결 : 항소] 【판시사항】 양도소득세 감면대상에서 제외되는 고급주택에 해당하는지 여부를 판단함에 있어 주상복합건축물의 건물 외벽 내부에 있는 발코니 면적을 전용면적에 포함시켜야 하는지 


# 1, Dense(FAISS) 리트리버 사용 준비 (임베딩 기반 (질의 임베딩 함수)

In [6]:
import faiss

def _normalize(mat):
    n = np.linalg.norm(mat, axis=1, keepdims=True) + 1e-12
    return mat / n

dim = embs.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(_normalize(embs).astype("float32"))

def dense_search_by_vec(query_vec, k=5):
    q = _normalize(query_vec[None, :].astype("float32"))
    D, I = index.search(q, k)
    return [(int(i), float(s)) for i, s in zip(I[0], D[0])]

from sentence_transformers import SentenceTransformer
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
_e5 = None

def e5_query_embed(q: str):
    global _e5
    if _e5 is None:
        _e5 = SentenceTransformer("intfloat/multilingual-e5-large-instruct", device=DEVICE)
        _e5.max_seq_length = 512
    qf = "query: " + q.strip()
    v = _e5.encode([qf], normalize_embeddings=True, show_progress_bar=False)[0]
    return v.astype("float32")

### 질문 시 질의 임베딩 생성 함수 = E5 instruct

---

# 2&3 키워드 리트리버 (2번 TF-IDF / 3번 BM25)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from rank_bm25 import BM25Okapi

def simple_tok(s):
    return s.lower().replace("\n"," ").split()

tfidf = TfidfVectorizer(max_features=100000, ngram_range=(1,2), min_df=2)
X = tfidf.fit_transform(texts)

def tfidf_search(q, k=5):
    sims = linear_kernel(tfidf.transform([q]), X)[0]
    idx = np.argsort(-sims)[:k]
    return [(int(i), float(sims[i])) for i in idx]

bm25 = BM25Okapi([simple_tok(t) for t in texts])

def bm25_search(q, k=5):
    scores = bm25.get_scores(simple_tok(q))
    idx = np.argsort(-scores)[:k]
    return [(int(i), float(scores[i])) for i in idx]

---
(추가)

## Wrapper

In [8]:
# RAGState style 반환용 wrapper retriever (추가해둠)

def retriever_dense(query, k=5):
    v = e5_query_embed(query)
    hits = dense_search_by_vec(v, k=k)
    docs = [{
        "doc_id": meta[i].get("doc_id"),
        "chunk_index": meta[i].get("chunk_index"),
        "score": score,
        "filename": meta[i].get("filename"),
        "text": texts[i]
    } for (i,score) in hits]
    return {"retrieved_docs": docs}

def retriever_tfidf(query, k=5):
    hits = tfidf_search(query, k=k)
    docs = [{
        "doc_id": meta[i].get("doc_id"),
        "chunk_index": meta[i].get("chunk_index"),
        "score": score,
        "filename": meta[i].get("filename"),
        "text": texts[i]
    } for (i,score) in hits]
    return {"retrieved_docs": docs}

def retriever_bm25(query, k=5):
    hits = bm25_search(query, k=k)
    docs = [{
        "doc_id": meta[i].get("doc_id"),
        "chunk_index": meta[i].get("chunk_index"),
        "score": score,
        "filename": meta[i].get("filename"),
        "text": texts[i]
    } for (i,score) in hits]
    return {"retrieved_docs": docs}

- 확인

In [9]:
q = "발코니 면적이 전용면적에 포함되는지 판례"
res = retriever_dense(q)
print(res["retrieved_docs"][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

{'doc_id': '판례_136542', 'chunk_index': 5, 'score': 0.9321610331535339, 'filename': '판례_136542.txt', 'text': '같이 이 사건 주택의 공부상 전용면적은 고급주택 기준을 충족하지 않으나, 공부상 전용면적에 발코니 면적을 포함할 경우 고급주택 기준을 충족하게 된다.\n따라서 이 사건의 쟁점은 고급주택 해당 여부를 판단함에 있어서 발코니 면적을 전용면적에 포함시켜야 하는지 여부에 있다.\n(2) 공동주택의 면적 산정에 관한 제 규정\n구 소득세법 시행령(2002. 10. 1. 대통령령 제17751호로 개정되기 전의 것) 제156조 에서는 고급주택의 범위와 관련하여 ‘전용면적’이라는 개념을 도입하였으나 이를 정의하고 있지는 않다.\n그런데 구 주택건설촉진법 시행령(2003. 11. 29. 대통령령 제18146호로 전문 개정되기 전의 것) 제11조의5 제1호 , 같은 법 시행규칙(2003. 12. 15. 건설교통부령 제382호로 전문 개정되기 전의 것) 제17조 제1항 및\n제2항\n에서는 ‘전용면적’이라는 개념을 도입하여 공동주택의 경우 외벽의 내부선을 기준으로 전용면적을 산정하되, 공용부분의 면적은 제외하도록 규정하는 한편, 전용면적 85㎡ 이하인 공동주택을 국민주택으로 분류하였다. 현행 주택법 제2조 제3호 에서도 마찬가지로 규정하고 있다.\n나아가 구 소득세법(2002. 12. 18. 법률 제6781호로 개정되기 전의 것) 제89조 제3호 , 같은 법 시행령(2002. 10. 1. 대통령령 제17751호로 개정되기 전의 것) 제156조 제2호 소정의 1세대 1주택 양도소득세 비과세, 법인세법 시행령 제92조의2 제2항 제1호의2 소정의 양도소득에 대한 과세특례, 소득세법 시행령 제112조 제1항 소정의 주택자금공제,\n제167조의3 제1항 제2호 (다)목\n소정의 1세대 3주택 배제대상, 조세특례제한법 시행령 제51조의2 소정의 자기관리부동산투자회사 과세특례,\n제106

## 조회 결과를 CSV로 저장

In [10]:
def run_all_retrievers(queries, top_k=5, out_csv="retriever_result.csv"):
    import csv
    rows = []

    # dense
    for q in queries:
        v = e5_query_embed(q)
        for rank,(i,s) in enumerate(dense_search_by_vec(v,k=top_k),1):
            rows.append({"retriever":"dense","query":q,"rank":rank,"score":s,
                         "doc_id":meta[i].get("doc_id"),
                         "chunk_index":meta[i].get("chunk_index"),
                         "filename":meta[i].get("filename"),
                         "snippet":texts[i][:200].replace("\n"," ")})

    # tfidf
    for q in queries:
        for rank,(i,s) in enumerate(tfidf_search(q,k=top_k),1):
            rows.append({"retriever":"tfidf","query":q,"rank":rank,"score":s,
                         "doc_id":meta[i].get("doc_id"),
                         "chunk_index":meta[i].get("chunk_index"),
                         "filename":meta[i].get("filename"),
                         "snippet":texts[i][:200].replace("\n"," ")})

    # bm25
    for q in queries:
        for rank,(i,s) in enumerate(bm25_search(q,k=top_k),1):
            rows.append({"retriever":"bm25","query":q,"rank":rank,"score":s,
                         "doc_id":meta[i].get("doc_id"),
                         "chunk_index":meta[i].get("chunk_index"),
                         "filename":meta[i].get("filename"),
                         "snippet":texts[i][:200].replace("\n"," ")})

    with open(out_csv,"w",newline="",encoding="utf-8") as f:
        writer = csv.DictWriter(f,fieldnames=rows[0].keys())
        writer.writeheader()
        writer.writerows(rows)
    print(f"저장 = {out_csv} ({len(rows)} rows)")

평가를 위한 여러 질문을 csv 폴더로 저장할 까 고민중.

## 각 설명

Dense(FAISS) → 임베딩 기반 검색 (E5 임베딩 + FAISS 벡터DB)

TF-IDF → 전통적인 키워드 가중치 검색 (단어 빈도 기반)

BM25 → 전통적인 키워드 검색 방식 중 하나, TF-IDF보다 검색 품질이 좋은 경우가 많음   


선택 이유  
- 연구 및 구현(RAG 포함)에서 **Dense + Sparse 비교**가 표준 실험 세트로 사용됨.
- 이를 기반으로 **Dense(BERT 계열) vs TF-IDF vs BM25**를 기본 비교 세트

---

## 우선 샘플 질문 리스트

In [11]:
queries = [
    "발코니 면적이 전용면적에 포함되는지 판례",
    "양도소득세 고급주택 요건 판례",
    "주상복합 건축물 전용면적 기준 판례",
    "증여세 과세 처분 취소 관련 판례",
    "청구기각 처분 사유가 된 판례",
    "취득세 부과처분 취소 사건 판례",
    "근저당권설정 비용 소득공제 여부 판례",
    "토지 양도차익 계산 관련 판례",
    "임대소득세 산정 방식 판례",
    "부당행위계산 부인 요건 판례",
    "세금 우편 송달 효력 다툰 판례",
    "형사판결 확정 후 행정소송 판례",
    "이중과세 여부 판단 기준 판례",
    "소멸시효 완성 주장 기각 판례",
    "기간 경과 후 증거 제출 허용 판례",
    "양도소득세 감면 대상 제외 사유 판례",
    "법인세 회계처리 기준 관련 판례",
    "상속세 신고 불이행 제재 판례"
]


run_all_retrievers(queries, top_k=5, out_csv="/content/drive/MyDrive/retriever_result.csv")

저장 = /content/drive/MyDrive/retriever_result.csv (270 rows)


## 결과 확인

In [12]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/retriever_result.csv")
df.head()

,retriever,query,rank,score,doc_id,chunk_index,filename,snippet
0,dense,발코니 면적이 전용면적에 포함되는지 판례,1,0.932161,판례_136542,5,판례_136542.txt,"같이 이 사건 주택의 공부상 전용면적은 고급주택 기준을 충족하지 않으나, 공부상 전..."
1,dense,발코니 면적이 전용면적에 포함되는지 판례,2,0.928897,판례_136542,10,판례_136542.txt,"체를 세우는 한편, 나아가 발코니에 유입되는 먼지와 찬 공기 등을 차단하기 위하여 ..."
2,dense,발코니 면적이 전용면적에 포함되는지 판례,3,0.925311,판례_136542,13,판례_136542.txt,다. 건축법 시행령 소정의 ‘노대’를 건축물 외부에 노출된 바닥형태의 구조물이라고 ...
3,dense,발코니 면적이 전용면적에 포함되는지 판례,4,0.922155,판례_136542,3,판례_136542.txt,"귀속 양도소득세 114,529,360원을 결정·고지하였다(이하 ‘이 사건 처분’이라..."
4,dense,발코니 면적이 전용면적에 포함되는지 판례,5,0.919304,판례_136542,7,판례_136542.txt,"닥면적에 산입하도록 규정하였고, 집합건물의 소유 및 관리에 관한 법률 제2조 제3호..."


# 해석
Dnese 기반의 리트리버 = (E5 임베딩 기반의 리트리버)  
"발코니 면적이 전용 면적에 포함되는 판례" 관련 질의 (0열의 답)  

- 판례_136542 문서의 5번 청크가 가장 관련 있다고 판단됐고 (유사도 0.93으로 1등으로 뽑혔음)  


즉 해당 질문에 대해서 E5 Dense 모델이 의미 기반 매칭을 하고 있는 것으로 판단된다고 생각할 수 있고,   


해당 내용에 대해서 성능 평가는 정답 데이터 셋으로 이후 진행되어야 함. (다음 파트)   


우선은 리트리버가 정상 작동함을 확인.

- 데이터 확인

## 현재까지 해둔 일  

3개의 리트리버 (Dense, tfidf, bm25)를 구현하여, 각 질의 마다 top-k의 결과를 결과(=doc_id)를 뽑아 csv로 저장.  

검색 결과 후보군 생성 단계를 만들어 두었습니다.